# Lab 4. Backpropogation
by Domrachev Ivan, B20-Ro-01

In [1]:
from nn_from_scratch.nodes import SoftMax, NormalizedSoftMax
import numpy as np

# Part 1. Softmax function

All the logic is implemented in the file [backprop.py](./backprop.py). There is an abstract class Node, which implements all the expected logic except for the function itself and the jacobian matrix calculation.

In [2]:
sm = SoftMax(5)

# Random input values (x itself, and assumed partial derivative)
x_input = np.array([0, 1, 2, 3, 4])
dL_dy = np.array([1, 2, 3, 2, 1])

# Forward call
y_value = sm.forward(x_input)

# Backpropogation
dL_dx = sm.backward(dL_dy)

y_value, dL_dx

(array([0.01165623, 0.03168492, 0.08612854, 0.23412166, 0.63640865]),
 array([-0.00510617,  0.01780491,  0.1345273 ,  0.13156147, -0.27878751]))

To validate the result, let's use the `sympy` for a numerical calculations:

In [3]:
import sympy as sp 

n = 5
x = sp.symbols(' '.join([f'x_{i}' for i in range(n)]))
exp_x = sp.Matrix([
    sp.exp(x_i) for x_i in x
])
exp_sum = sum(exp_x)

softmax = exp_x / exp_sum
softmax

Matrix([
[exp(x_0)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))],
[exp(x_1)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))],
[exp(x_2)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))],
[exp(x_3)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))],
[exp(x_4)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))]])

In [4]:
softmax_jac = softmax.jacobian(x)
softmax_jac

Matrix([
[exp(x_0)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4)) - exp(2*x_0)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))**2,                                                           -exp(x_0)*exp(x_1)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))**2,                                                           -exp(x_0)*exp(x_2)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))**2,                                                           -exp(x_0)*exp(x_3)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))**2,                                                           -exp(x_0)*exp(x_4)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))**2],
[                                                          -exp(x_0)*exp(x_1)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))**2, exp(x_1)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4)) - exp(2*x_1)/(exp(x_0) + exp(x_1) + exp(x_2) + exp(x_3) + exp(x_4))**2,                                     

In [5]:
softmax_lambda = sp.lambdify(x, softmax)
jacobian_lambda = sp.lambdify(x, softmax_jac)

y_value_sp = softmax_lambda(*x_input).ravel()
dL_dx_sp = (jacobian_lambda(*x_input) @ dL_dy).ravel()

assert ((y_value_sp - y_value)<1e-3).all()
assert ((dL_dx_sp - dL_dx)<1e-3).all()

Also, `SoftMax` supports matrix inputs. It will apply `SoftMax` rowwise

In [6]:
sm = SoftMax((2, 5))

# Random input values (x itself, and assumed partial derivative)
x_input = np.array([
    [0, 1, 2, 3, 4,],
    [0, 1, 2, 3, 4]
])
dL_dy = np.array([
    [1, 2, 3, 2, 1], 
    [1, 2, 3, 2, 1]
])

# Forward call
y_value = sm.forward(x_input)

# Backpropogation
dL_dx = sm.backward(dL_dy)

for i in range(2):
    assert ((y_value_sp - y_value[i])<1e-3).all()
    assert ((dL_dx_sp - dL_dx[i])<1e-3).all()

# Part 2. Normalized SoftMax

The forward pass of normalized SoftMax is trivial to implement. Regarding jacobian, it could be expressed as follows:
$$\frac{\partial y}{\partial x} = \frac{\partial y}{\partial x^{norm}}\frac{\partial x^{norm}}{\partial x}$$

where term $\frac{\partial y}{\partial x^{norm}}$ is a softmax function of $x^{norm}$ term, and $\frac{\partial x^{norm}}{\partial x}$ structure depends on the maximal element idx

One could see, that the only differences are:
1. Denominator everywhere is multiplied by a maximal element
2. The differentiation over maximal element contains contains an additional term
3. The diagonal element, corresponding to the maximal element contains additional term too

Using that, it's easy to compute jacobian numerically

In [7]:
sm = NormalizedSoftMax(5)

# Random input values (x itself, and assumed partial derivative)
x_input = np.array([0, 1, 4, 2, 3])
dL_dy = np.array([1, 2, 3, 2, 1])

# Forward call
y_value = sm.forward(x_input)

# Backpropogation
dL_dx = sm.backward(dL_dy, reset_after=False)

y_value, dL_dx

(array([0.11405072, 0.14644403, 0.31002201, 0.18803785, 0.24144538]),
 array([ 0.02596599,  0.04249374, -0.00692263,  0.01930594, -0.08084303]))

To validate the result, let's use the `sympy` for a numerical calculations:

In [8]:
n = 5
x = sp.symbols(' '.join([f'x_{i}' for i in range(n)]))
x_norm = sp.Matrix([x_i / x[2] for x_i in x])
exp_x_norm = sp.Matrix([
    sp.exp(x_norm_i) for x_norm_i in x_norm
])
exp_sum_norm = sum(exp_x_norm)

# Assume that 2th element is the biggest
softmax_norm = exp_x_norm / exp_sum_norm
softmax_norm_jac = softmax_norm.jacobian(x)

In [9]:
softmax_lambda = sp.lambdify(x, softmax_norm)
jacobian_lambda = sp.lambdify(x, softmax_norm_jac)

y_value_sp = softmax_lambda(*x_input).ravel()
dL_dx_sp = (jacobian_lambda(*x_input) @ dL_dy).ravel()

assert ((y_value_sp - y_value)<1e-3).all
assert ((dL_dx_sp - dL_dx)<1e-3).all()

Also, `NormalizedSoftMax` supports matrix inputs. It will apply `NormalizedSoftMax` rowwise

In [10]:
sm = NormalizedSoftMax((2, 5))

# Random input values (x itself, and assumed partial derivative)
x_input = np.array([
    [0, 1, 4, 2, 3],
    [0, 1, 4, 2, 3]
])
dL_dy = np.array([
    [1, 2, 3, 2, 1], 
    [1, 2, 3, 2, 1]
])

# Forward call
y_value = sm.forward(x_input)

# Backpropogation
dL_dx = sm.backward(dL_dy)

for i in range(2):
    assert ((y_value_sp - y_value[i])<1e-3).all()
    assert ((dL_dx_sp - dL_dx[i])<1e-3).all()